<a href="https://colab.research.google.com/github/Sammylayo/SammyPortfolioProject/blob/main/TensorFlow_basic_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses

dataset = tf.keras.utils.get_file("aclImdb_v1", url, untar=True, cache_dir='.', cache_subdir=''): The tf.keras.utils.get_file() function is used to download and extract the dataset. It has several arguments:

"aclImdb_v1": This is the name that will be given to the downloaded file. It serves as the base name for the file when it is saved to disk.

url: The URL of the dataset to be downloaded.

untar=True: The untar argument is set to True, indicating that the downloaded file should be extracted. Since the dataset is in the form of a .tar.gz file, it will be extracted after downloading.

cache_dir='.': The cache_dir argument specifies the directory where the downloaded file will be stored. In this case, '.' represents the current working directory.

cache_subdir='': The cache_subdir argument specifies a subdirectory within the cache directory where the file will be saved. In this case, an empty string means that the file will be saved directly in the cache directory without any subdirectory.

The dataset variable will hold the path to the downloaded and extracted dataset.

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb'): The os.path.join() function is used to create the path to the directory where the dataset is extracted. It combines the parent directory of the dataset file (i.e., the cache directory) with the subdirectory 'aclImdb'. The resulting dataset_dir variable will contain the path to the directory where the IMDb movie reviews dataset is extracted

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 [==============================] - 12s 0us/step


In [3]:
os.path.dirname(dataset) ##which is the current directory

'.'

In [4]:
os.listdir(dataset_dir)

['train', 'test', 'README', 'imdbEr.txt', 'imdb.vocab']

In [12]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['neg',
 'urls_unsup.txt',
 'pos',
 'urls_pos.txt',
 'urls_neg.txt',
 'unsupBow.feat',
 'labeledBow.feat']

In [10]:
pos_dir = os.path.join(train_dir, 'pos')
text = os.listdir(pos_dir)[0]
text

'1769_8.txt'

In [7]:
sample_file = os.path.join(pos_dir, text)
with open(sample_file) as sammy:
  print(sammy.read())

Just came out of a sneak preview for this film. It had me laughing every 30 secs. The ending was so funny that tears were rolling down my face and it had me wishing I hadn't bought that large coke. There are definitely some lulls, but, overall, highly entertaining. The movie lets Steve Carell have a chance to shine after stealing the spotlight from both Jim Carrey in "Bruce Almighty" and Will Ferrell "Anchorman: The Legend of Ron Burgendy" in their movies. Paul Rudd is hilarious as always. I love that he can be so funny in these broad comedies and continues to work in indie dramas (like P.S.). I think that Seth Rogen should be getting more work, because he so freaking talented and engaging. Leslie Mann also had some incredibly funny moments. I highly recommend it for those who just want to laugh like a maniac. However, if you're easily offended, don't see this movie. If you're a rabid feminist, don't see this movie. And, please, not matter what, even if you think you're one of those "h

In [11]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [13]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train/',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [16]:
print(list(raw_train_ds.as_numpy_iterator())[0][0][0].strip())

b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />There\'s a mysteri

In [20]:
len(next(iter(raw_train_ds))[0])

32

In [51]:
list(next(iter(raw_train_ds)))[1][2]

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [39]:
type(raw_train_ds)

tensorflow.python.data.ops.batch_op._BatchDataset

In [45]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(1):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'Skippy from "Family Ties" plays Eddie, a wussy \'metal\' nerd who gets picked on. When his favorite wussy \'metal\' singer, Sammi Curr, dies, he throws a hissy fit tearing down all the posters on his bedroom wall. But when he later gets an unreleased record that holds the spirit of his dead \'metal\' idol. He first gets sucked into ideas of revenge, but then he doesn\'t want to take it as far as Sammi does. Which isn\'t really that far as his main victims only seem to go to the hospital. This movie is utterly laughable and has about as much to do with real metal as say, "Rock Star". OK, maybe a tad more than that piece of junk, but you get my point. And how ANYone can root for a guy played by Skippy from "Family Ties" I haven\'t a clue. The cameo by Gene Simmons is OK, and Ozzy Osbourne reaches coherency, I applaud him for that, but otherwise skip this one.<br /><br />My Grade: D <br /><br />Eye Candy:Elise Richards gets topless, an a topless extra at a pool party'
Label 0


In [52]:
len(text_batch)

32

In [53]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [54]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [55]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


The custom_standardization function takes input_data as an argument, which represents a text string or a tensor of text data. The function applies the following operations for text standardization:

Lowercasing: The tf.strings.lower() function is used to convert the text to lowercase. This helps normalize the text by removing case distinctions.

Stripped HTML: The tf.strings.regex_replace() function is used to replace the <br /> HTML tag with a space. This is typically done to handle HTML content where <br /> represents line breaks.

Punctuation Removal: The tf.strings.regex_replace() function is again used to remove punctuation marks from the text. The regular expression [%s]' % re.escape(string.punctuation) matches any punctuation characters defined in the string.punctuation module from the text and replaces them with an empty string.

The function returns the processed text tensor after applying these standardization operations.

You can use this custom_standardization function as a preprocessing step for text data in your TensorFlow models, such as tokenization or embedding layers, to ensure consistent and standardized input representation.

In [56]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

standardize: This parameter refers to a function that performs custom text standardization or preprocessing. In the provided code, custom_standardization is passed as the value. This function is applied to each input text before tokenization and vectorization.

max_tokens: This parameter determines the maximum number of tokens (unique words) to consider when tokenizing the text data. In this case, max_features is used as the value, indicating that the tokenizer should consider the top 10,000 most frequent tokens.

output_mode: This parameter specifies the output mode of the TextVectorization layer. The value 'int' indicates that the layer should output integer representations of the tokens.

output_sequence_length: This parameter determines the fixed sequence length for the output tensor. It specifies the number of tokens in each sequence by padding or truncating the input text. In this case, sequence_length is set to 250, indicating that each input sequence will have a length of 250 tokens.

By creating the TextVectorization layer with these configurations, you can use it to preprocess and vectorize text data in your TensorFlow models. The layer standardizes the text, tokenizes it, converts the tokens to integers, and ensures a consistent sequence length for the input data.

In [57]:
max_features = 15000
sequence_length = 500

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length, )

train_text = raw_train_ds.map(lambda x, y: x): This line of code creates a new dataset train_text by mapping the raw_train_ds dataset using a lambda function. The lambda function (lambda x, y: x) extracts only the text (x) from each element of the raw_train_ds dataset, discarding the labels (y).

vectorize_layer.adapt(train_text): This line adapts the vectorize_layer to the train_text dataset. The adapt() method analyzes the dataset and adjusts the internal state of the layer accordingly. This adaptation process enables the layer to build the vocabulary of tokens based on the text in the training dataset.

By calling adapt() on the vectorize_layer with the train_text dataset, the layer learns the vocabulary of tokens and their indices. It determines the unique tokens present in the training text data, assigns indices to them, and sets up the necessary internal mappings to convert text data to vectorized representations.

After adapting the vectorize_layer, it can be used to preprocess and vectorize text data consistently and effectively in subsequent stages, such as model training or evaluation.

In [58]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

text = tf.expand_dims(text, -1): This line of code uses tf.expand_dims() to add an extra dimension to the text tensor. This is done to match the expected input shape of the vectorize_layer. The -1 argument indicates adding the new dimension as the last dimension of the tensor.

return vectorize_layer(text), label: This line returns the result of applying the vectorize_layer to the expanded text tensor, along with the original label. The vectorize_layer(text) call applies the text vectorization process defined by the TextVectorization layer to the text tensor, converting it into a vectorized representation. The resulting tensor is returned along with the original label.

By using this vectorize_text function, you can conveniently preprocess and vectorize text data along with their corresponding labels in a consistent manner. This function is typically used when preparing input data for training or evaluation in a TensorFlow model.

In [59]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

next(iter(raw_train_ds)): This line uses the iter() function to create an iterator from the raw_train_ds TensorFlow Dataset object. The next() function is then called to retrieve the next element from the iterator, which corresponds to a batch of samples.

In [ ]:
exp_no = tf.expand_dims(no, 2)#The 2nd positional argument determines where the expansion will take place... note that the 2 in this case causes the shape to have 1 in the 3rd position...
print(exp_no)

tf.Tensor(
[[[1.]
  [1.]]

 [[1.]
  [1.]]], shape=(2, 2, 1), dtype=float32)


In [60]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review.numpy())
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review b'Uta Hagen\'s "Respect for Acting" is the standard textbook in many college theater courses. In the book, Hagen presents two fundamentally different approaches to developing a character as an actor: the Presentational approach, and the Representational approach. In the Presentational approach, the actor focuses on realizing the character as honestly as possible, by introducing emotional elements from the actor\'s own life. In the Representational approach, the actor tries to present the effect of an emotion, through a high degree of control of movement and sound.<br /><br />The Representational approach to acting was still partially in vogue when this Hamlet was made. British theater has a long history of this style of acting, and Olivier could be said to be the ultimate king of the Representational school.<br /><br />Time has not been kind to this school of acting, or to this movie. Nearly every working actor today uses a Presentational approach. To the modern eye, Olivier\'s 

In [61]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  silent
 313 --->  night
Vocabulary size: 15000


In [62]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE: AUTOTUNE is a special value in TensorFlow's tf.data module that allows the system to automatically tune the buffer size based on available system resources. It ensures optimal performance by dynamically adjusting the buffer size for reading and processing data.

train_ds.cache(): The cache() method caches the training dataset in memory after the first epoch. This caching helps avoid reading and preprocessing the training data from disk in each epoch, significantly improving the training performance for subsequent epochs.

train_ds.prefetch(buffer_size=AUTOTUNE): The prefetch() method prefetches data from the training dataset. It allows the data loading and preprocessing steps to overlap with model training, utilizing the available CPU/GPU resources efficiently. The buffer_size parameter specifies the number of elements to prefetch, and AUTOTUNE is used to automatically determine an appropriate buffer size based on the available system resources.

In [63]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

layers.Embedding(max_features + 1, embedding_dim): The first layer in the model is an Embedding layer. This layer is responsible for learning word embeddings, which are dense vector representations of words in the input text. The max_features + 1 is the input size, representing the vocabulary size, and embedding_dim is the dimensionality of the word embeddings.

layers.Dropout(0.2): The first dropout layer is used to prevent overfitting by randomly setting a fraction of input units to zero during training. The dropout rate is set to 0.2, meaning 20% of the input units will be randomly dropped during training.

In [64]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)], )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          240016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 240033 (937.63 KB)
Trainable params: 240033 (937.63 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [65]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [66]:
epochs = 5
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1)

Epoch 1/5
625/625 [==============================] - 12s 17ms/step - loss: 0.6792 - binary_accuracy: 0.6449 - val_loss: 0.6543 - val_binary_accuracy: 0.6970
Epoch 2/5
625/625 [==============================] - 8s 13ms/step - loss: 0.6045 - binary_accuracy: 0.7742 - val_loss: 0.5633 - val_binary_accuracy: 0.7990
Epoch 3/5
625/625 [==============================] - 7s 12ms/step - loss: 0.5112 - binary_accuracy: 0.8245 - val_loss: 0.4840 - val_binary_accuracy: 0.8374
Epoch 4/5
625/625 [==============================] - 7s 11ms/step - loss: 0.4381 - binary_accuracy: 0.8532 - val_loss: 0.4258 - val_binary_accuracy: 0.8524
Epoch 5/5
625/625 [==============================] - 8s 13ms/step - loss: 0.3849 - binary_accuracy: 0.8719 - val_loss: 0.3844 - val_binary_accuracy: 0.8642


In [67]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 13s 16ms/step - loss: 0.3944 - accuracy: 0.8596
0.8595600128173828


In [68]:
examples = [
  "The movie was very great wonderful!",
  "The movie was okay.",
  "The movie was very terrible...",
  "The movie was the best film ever made",
  "horrible"
]

export_model.predict(examples)

1/1 [==============================] - 0s 154ms/step


array([[0.6051175 ],
       [0.4705807 ],
       [0.4675867 ],
       [0.51737005],
       [0.46485427]], dtype=float32)